# Linearna regresija

Jedan od prvih algoritama za nalaženje trenda među podacima je ***Linearna regresija***. Ovaj algoritam nadgledanog učenja, unatoč postojanju nekolicine kvalitetnijih algoritama u istoj oblasti, uglavnom zbog svoje jednostavnosti nalazi široku primjenu u praksi.

Trend podataka je, grubo rečeno, patern po kom se podaci ponavljaju za dati input. Npr. ukoliko primijetimo da se cijena neke konkretne tipe auta (npr. BMW m3 e46) smanjuje sa brojem pređenih kilometara, kažemo da je cijena te tipe auta opadajućeg trenda u odnosu na input **broj pređenih kilometara**. Recimo još da smo se domogli stvarnih podataka u kojima imamo informaciju o različitim cijenama date tipe auta za respektivan broj pređenih kilometara. Analizom parova (broj pređenih kilometara, cijena auta) možemo "izračunati" i na koji način cijena auta ovisi o broju pređenih kilometara. Konkretno u ovom slučaju možemo izračunati kojom "brzinom" opada cijena auta rastom broja pređenih kilometara. Na taj način kažemo da smo izračunali (ili često rečeno, uhvatili) trend po kojim se cijena auta mijenja u odnosu na broj pređenih kilometara i time možemo za bilo koji broj pređenih kilometara, o kom nemamo informaciju u skupu podataka kog smo se domogli, izračunati (uglavnom aproksimativno) cijenu auta. Ovaj način aproksimativnog računanja nekog podatka se naziva **predikcijom** datog podatka, a **način** kojim "računamo" isti se naziva **modelom predikcije** ili **modelom učenja** ili **modelom mašinskog učenja**. Tako da možemo kazati da je **regresija** upravo jedan model mašinskog učenja i to tačnije, model **nadgledanog** mašinskog učenja. Rroces kroz koji smo "izračunali" zavisnost između ulaznih i izlaznih podataka modela, konkretno u primjeru iznad, između cijene auta i broja pređenih kilometara, se naziva **treningom** ili **obučavanjem** modela nad datim skupom podataka, a dati skup podataka, **trening skupom** podataka.

Prisjetimo se (sa predavanja) da je element trening skupa, bilo kog modela nadgeldanog mašinskog učenja, formata ($\overline{x}$, $\overline{y}$) gdje $\overline{x}$ odgovara ulaznom, a $\overline{y}$ izlaznom podatku modela. *Svaki model nadgledanog mašinskog učenja, u suštini, nije ništa do jedna **funkcija** koja prima određene podatke kao input i na osnovu tih podataka računa i vraća neki rezultat.*

Recimo da imamo na raspolaganju sljedeći trening skup: {(10000, 31000), (400000, 19000), (5000, 32000), (0, 40000), (1000, 33000), (100000, 26000), (50000, 29000), (50, 35000), (20000, 30000), (200000, 20000)}, gdje u svakom elementu skupa $(x, y)$, $x$ odgovara pređenom broju kilometara, a $y$ cijeni nekog auta. Smjestimo dati trening skup u niz `car_mileage_vs_value_data_set`:

In [2]:
from typing import List, Tuple

car_mileage_vs_value_data_set: List[List[int]] = [
    (10000, 31000), (400000, 19000), (5000, 32000), (0, 40000),
    (1000, 33000), (100000, 26000), (50000, 29000), (50, 35000),
    (20000, 30000), (200000, 20000)]

Konvertujmo dati niz u *numpy* niz radi lakše manipulacije istim:

In [3]:
import numpy as np

car_mileage_vs_value_data_set: np.ndarray = np.array(car_mileage_vs_value_data_set)

Iscrtajmo dati trening skup uz napomene ispod:
1. Koristimo biblioteku za vizualizaciju `plotly`.
2. Naredba `ply.init_notebook_mode(connected=True)` daje do znanja biblioteci Plotly da se sva iscrtavanja vrše u Jupyter Notebook okruženju.
2. `car_mileage_vs_value_data_set` je sada dvodimenzionalni *numpy* niz.
3. `car_mileage_vs_value_data_set.T` vraća transponovanu matricu `car_mileage_vs_value_data_set`.
4. `car_mileage_vs_value_data_set.T[0]` je jednodimenzionalni niz vrijednosti pređenih kilometara auta.
5. `car_mileage_vs_value_data_set.T[1]` je jednodimenzionalni niz vrijednosti cijena auta.
6. `go.Scatter()` formira objekat iz biblioteke Plotly koji čuva informaciju u podacima koje treba iscrtati kao i o formatu iscrtavanja istih.
7. Slično je i sa `go.Layout()`. Formira se objekat koji čuva informaciju o formatu iscrtavanja plana (layout-a) cijelog plota (bez podataka).
8. Objekat `go.Figure` objedinjuje informacije u *Scatter* i *Layout* objektu i naredba *ply.iplot()* vrši samo iscrtavanje.
9. `typehint` se po konvenciji ne koristi kod instanciranja klasa.

In [11]:
import plotly.offline as ply
import plotly.graph_objs as go

ply.init_notebook_mode(connected=True)

car_mileages: np.ndarray = car_mileage_vs_value_data_set.T[0]
car_values: np.ndarray = car_mileage_vs_value_data_set.T[1]

plot_data = go.Scatter(x=car_mileages, 
                       y=car_values, 
                       mode='markers',
                       marker=dict(color='black'))

layout = go.Layout(xaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

figure = go.Figure(data=[plot_data], layout=layout)

ply.iplot(figure)

Intuitivno, na navedenom primjeru, ideja linearne regresije je sadržana u pronalasku prave $p: y = m_1\cdot x + m_0$ takve da je vrijednost funkcije greške date prave nad datim trening skupom što manja *(pogledati poglavlje o Regresionoj analizi u profesorovoj skripti za više informacija u funkcijama greške)*. Koristiti ćemo *mean squared error* kao funkciju greške. Dakle želimo da pronađemo parametre $m_1$ i $m_2$ takve da vrijednost $\frac{1}{|\chi|}\cdot\sum_{(x, y) \in \chi}{(m_1\cdot x + m_0 - y)^2}$ bude što manja, gdje je $\chi$ naš trening skup. Drugim riječima, želimo da pronađemo položaj prave $p$ takav da je, za sve parove $(x_i, y_i)$ trening skupa, (kumulativno) odstupanje vrijednosti $m_1\cdot x_i + m_0$ od $y_i$ što manja.

Kada pronađemo takve parametre, nazovimo ih $m_1^*$ i $m_0^*$, to jeste pravu $p^*: y = m_1^*\cdot x + m_0^*$, možemo za bilo koji input, broj pređenih kilometara, aproksimativno izračunati cijenu koju bi jedno takvo auto moglo imati, jednostavno uvrstivši željeni broj pređenih kilometara $x$ u izraz $m_1^*\cdot x + m_0^*$. Drugim riječima, cijena auta se kreće po pravoj $p^*$ u odnosu na broj pređenih kilometara. U posljednjoj rečenici se oslikava i glavna osobina jednodimenzionalne linearne regresije (tj. linearne regresije sa samo jednim inputom, kao u našem primjeru. Kasnije ćemo se upoznati i sa višedimenzionalnom regresijom), a to je da **vrijednost output-a se kreće po pravoj $p*$ u odnosu na input i to postavljenoj u takav položaj da minimizira kumulativnu grešku (odstupanje) u odnosu na dati trening skup**.

Na primjeru ispod se nalazi navedena prava $p^*$ izračunata gotovom metodom `linear_model` biblioteke **sklearn**. Ne zamarajmo se toliko samom bibliotekom **sklearn** i njenim metodama sada, ali obratimo pažnju na to kako računamo *Mean squared error*.

In [12]:
from sklearn import linear_model

# np.expand_dims(a, axis=1) for input of a = [1, 2, 3] returns [[1], [2], [3]]
# This expanded format of np array is required for an input to linear_model.LinearRegression.fit method
# as well as for linear_model.LinearRegression.predict method
expanded_car_mileages: np.ndarray = np.expand_dims(car_mileages, axis=1)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training set
regr.fit(expanded_car_mileages, car_values)

# The mean squared error
print(f'Mean squared error: {np.mean((regr.predict(expanded_car_mileages) - car_values) ** 2)}')

linear_regression_line = go.Scatter(x=car_mileages, 
                                    y=regr.predict(expanded_car_mileages),
                                    mode='lines',
                                    line=dict(color='blue', width=3))

figure = go.Figure(data=[plot_data, linear_regression_line], layout=layout)

ply.iplot(figure)

Mean squared error: 10080774.695512509


Sada ukoliko bismo željeli da procijenimo vrijednost auta koju bi isto moglo imati na kilometražama od 150000km, 250000km i 300000km, dovoljno je da izračunamo visinu prave za date inpute:

In [18]:
test_mileages: np.ndarray = np.array([150000, 250000, 300000])
expanded_test_mileages: np.ndarray = np.expand_dims(test_mileages, axis=1)
approximated_values: np.ndarray = regr.predict(expanded_test_mileages)

for mileage, value in zip(test_mileages, approximated_values):
    print(f'Mileage: {mileage} -> Value: {round(value)}')

Mileage: 150000 -> Value: 26463.0
Mileage: 250000 -> Value: 22209.0
Mileage: 300000 -> Value: 20082.0


Nameće se pitanje, na koji način možemo pronaći jednu takvu pravu $p^*$.

Postoje trenutci u praksi kada nam je potrebna brza, ali dovoljno dobra analiza podataka (tačnije analiza trenda podataka). Npr. prilikom pisanja poslovnih prijedloga ili POC-a *(Proof of concept)* projekata (i to ne nužno samo Data Science projekata). Zatim često je potreba plitka analiza podataka prije nego li se odlučimo za korištenje nekog kompleksnijeg algoritma nadgledanog učenja itd.